# Conversão de arquivos .zip para .csv

Os dados utilizados para o projeto são disponibilizados originalmente como diversos arquivos .txt. Estes arquivos foram baixados como dois arquivos .zip, um contendo apenas textos referentes a notícias falsas (fake.zip) e outro referente a notícias verdadeiras (true.zip). Os dois arquivos foram subidos para um bucket S3 na AWS e também estão disponíveis neste repositório, no diretório full_texts.

Para iniciar o processamento de dados, os arquivos .zip armazenados no S3 são abertos e seus conteúdos são armazenados em um DataFrame de duas colunas: 'fake' (1 se a notícia for falsa, 0 se for verdadeira) e 'text' (texto da notícia). Inicialmente são carregadas todas as notícias falsas e, em seguida, as notícias reais, de forma que as labels ficam concentradas no DataFrame, com todas as notícias falsas nos primeiros índices e todas as notícias reais concentradas nos últimos índices. Para evitar qualquer enviesamento, o DataFrame então tem suas linhas embaralhadas.

Finalmente, um arquivo csv ('dados_brutos.csv') é salvo com estes dados, para que possa ser utilizado na análise exploratória e pré-processamento de dados. Este é mantido tanto no presente repositório e uma cópia é também subida ao mesmo bucket da AWS utilizado no início do procedimento.

In [12]:
from zipfile import ZipFile
import pandas as pd
import os
import io
import s3fs
import boto3
fs = s3fs.S3FileSystem()

class DataKeys:
    fake = 'fake'
    text = 'text'

data_dict = {DataKeys.fake: [], DataKeys.text: []}

In [3]:
bucket_name = 'projetointerdisciplinardadosbrutos'
bucket_path = f's3://{bucket_name}/'
fake_path = bucket_path + 'fake.zip'
true_path = bucket_path + 'true.zip'

paths_tuples = [(fake_path, 1), (true_path, 0)]

In [4]:
for s3_path, is_fake in paths_tuples:
    with fs.open(s3_path) as zip_path:
        with ZipFile(zip_path, 'r') as zip:
            for file in zip.namelist():
                if file.endswith('txt'):
                    with io.TextIOWrapper(zip.open(file), encoding="utf-8") as f:
                        file_text = f.read()
                        data_dict[DataKeys.fake].append(is_fake)
                        data_dict[DataKeys.text].append(file_text)

In [9]:
data_df = pd.DataFrame.from_dict(data_dict)
data_df = data_df.sample(frac=1).reset_index(drop=True)
data_df.head(20)

,fake,text
0,0,Maranhão passa por cima de convocação de líder...
1,1,Advogado de Lula diz que TRIPLEX é imóvel popu...
2,1,"Bolsonaro manda recado duro para a Globo: ""A f..."
3,0,"Segunda-feira, 26 de fevereiro . Bom dia! Aqui..."
4,0,TRF4 encerra 2017 mantendo histórico de aument...
5,1,Boechat e Jornal da Band desmascaram o governo...
6,0,Saiba como os políticos reagiram à denúncia co...
7,1,"Dirceu convoca militância para o ""dia da revol..."
8,1,"Monstro! Em Manaus, médico foi flagrado (no mo..."
9,1,O dia que Jô elogiou Fidel Castro e tomou uma ...


In [ ]:
csv_name = 'dados_brutos.csv'
data_df.to_csv(csv_name, encoding='utf-8')

In [ ]:
s3 = boto3.resource('s3')
with open(csv_name, 'b') as f:
    s3.Bucket(bucket_name).put_object(Key=csv_name, Body=b)